In [26]:
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd

### The cell below gets all the coin names from the home page and stores them as an `id` instead of being limit requested by the API.

In [27]:
page = requests.get('http://coinmarketcap.com/all/views/all/')
tree = html.fromstring(page.content)

trs = tree.xpath('//table[@id="currencies-all"]/tbody/tr')
coins = []
for tr in trs:
    coins.append(tr.attrib.get('id').replace("id-",""))

In [28]:
coins

['bitcoin',
 'ethereum',
 'ripple',
 'bitcoin-cash',
 'eos',
 'stellar',
 'litecoin',
 'tether',
 'cardano',
 'monero',
 'tron',
 'iota',
 'dash',
 'binance-coin',
 'neo',
 'ethereum-classic',
 'nem',
 'tezos',
 'vechain',
 'dogecoin',
 'zcash',
 'omisego',
 'bitcoin-gold',
 'maker',
 'bytecoin-bcn',
 'ontology',
 'lisk',
 '0x',
 'decred',
 'qtum',
 'bitcoin-diamond',
 'bitshares',
 'nano',
 'zilliqa',
 'digibyte',
 'siacoin',
 'icon',
 'steem',
 'aeternity',
 'verge',
 'waves',
 'pundi-x',
 'bytom',
 'basic-attention-token',
 'electroneum',
 'metaverse',
 'holo',
 'stratis',
 'golem-network-tokens',
 'augur',
 'status',
 'komodo',
 'populous',
 'trueusd',
 'cryptonex',
 'chainlink',
 'waltonchain',
 'reddcoin',
 'ardor',
 'mithril',
 'wanchain',
 'kucoin-shares',
 'iostoken',
 'maidsafecoin',
 'eternal-token',
 'moac',
 'aion',
 'hypercash',
 'aurora',
 'digixdao',
 'loopring',
 'huobi-token',
 'aelf',
 'bancor',
 'dentacoin',
 'cybermiles',
 'rchain',
 'decentraland',
 'gxchain',
 'd

### The cell below iterates through all the coins from the home page and scrapes their historical data into a CSV file - this procedure should be performed once daily or in script form this can be written in such a way that a jupyter notebook on google scripts is run once daily to update the historical data files

In [25]:
for index in range(750,len(coins)):
    url = "https://coinmarketcap.com/currencies/" + coins[index] + "/historical-data/?start=20130428&end=20300802"
    content = requests.get(url).content
    soup = BeautifulSoup(content,'html.parser')
    table = soup.find('table', {'class': 'table'})

    data = [[td.text.strip() for td in tr.findChildren('td')]
            for tr in table.findChildren('tr')]
    
    df = pd.DataFrame(data)
    df.drop(df.index[0], inplace=True) # first row is empty
    df[0] =  pd.to_datetime(df[0]) # date

    for i in range(1,7):    
        df[i] = pd.to_numeric(df[i].str.replace(",","").str.replace("-","")) # some vol is missing and has -
    
    df.columns = ['Date','Open','High','Low','Close','Volume','Market Cap']
    df.set_index('Date',inplace=True)
    df.sort_index(inplace=True)
    df.to_csv(coins[index] +'.csv', encoding='utf-8')

AttributeError: 'NoneType' object has no attribute 'findChildren'